Primeiro vamos definir a classe Dataset.
Ela possui três métodos atualmente: gerarTreinamento, gerarTeste e gerarValidacao.
Ambos os métodos retornam um par (textos, notas).
O texto textos[i] tirou a nota notas[i]. 
textos[i] é uma lista de listas: um texto é uma lista de parágrafos.
A competencia a ser avaliada é passada como parametro na criação do Dataset --- são 5 competencias (de 0 a 4), cada uma varia de 0 a 5 pontos

In [1]:
from build_dataset import Corpus
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_error
from skll import metrics as m
import numpy as np
from sklearn import metrics
import sklearn

class Dataset:
    def __init__(self, competence):
        self.c = Corpus()
        self.c.read_corpus().shape
        self.train, self.valid, self.test = self.c.read_splits()
        #print(self.train.shape)
        self.train.loc[1:5, ['essay', 'score', 'competence']] 
        #print(self.valid.shape)
        self.valid.loc[1:5, ['essay', 'score', 'competence']]
        #print(self.test.shape)
        self.test.loc[1:5, ['essay', 'score', 'competence']]
        self.competence = competence
    def UnirListas(self, lista):
        unificado = ""
        unificado = unificado+lista[0]+'\n'
        for t in range(1,len(lista)):
            unificado += lista[t]+'\n'
        return unificado
        
    def gerarTreinamento(self):
        """
        retorna um conjunto (texto, nota_competencia1). Um texto eh composto de varios paragrafos, cada paragrafo eh uma lista 
        """
        textos = []
        notas = []
        for index, row in self.train.iterrows():
            texto = self.UnirListas(row['essay'])
            notas.append( float(row['competence'][self.competence] / 40))
            textos.append(texto)
        return textos, notas
    def gerarTeste(self):
        textos = []
        notas = []
        for index, row in self.test.iterrows():
            texto = self.UnirListas(row['essay'])
            notas.append( float(row['competence'][self.competence] / 40))
            textos.append(texto)
        return textos, notas
    def gerarValidacao(self):
        textos = []
        notas = []
        for index, row in self.valid.iterrows():
            texto = self.UnirListas(row['essay'])
            notas.append( float(row['competence'][self.competence] / 40))
            textos.append(texto)
        return textos, notas

In [2]:
ds = Dataset(1)
texto_treinamento, nota_treinamento = ds.gerarTreinamento()
texto_teste, nota_teste = ds.gerarTeste()
texto_valid, nota_valid = ds.gerarValidacao()

Agora vamos importar o Tokenizer (o mesmo do BERT que usaremos), definiremos para usar a GPU se possível e definiremos a rede neural.

A rede neural é: aplicar o BERT sobre a entrada gerando um vetor 1x768, por fim, uma camada densa de 768x1.

In [3]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModel

#modelo = 'xlm-roberta-large'
modelo = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(modelo,model_max_length=512, truncation=True, do_lower_case=False)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
pesos = sklearn.utils.class_weight.compute_class_weight('balanced', classes=[0.0,1.0,2.0,3.0,4.0,5.0], y=nota_treinamento)
pesos = torch.tensor(pesos).float().to("cuda")
print(pesos)

cuda
tensor([7.2027, 8.8833, 1.1151, 0.4434, 0.4453, 2.8503], device='cuda:0')


In [4]:
def TransformarTextoEmInput(textos):
    tokenizados = []
    for indice in range(len(textos)):
            tokens = tokenizer.encode_plus(textos[indice], add_special_tokens=True, truncation=True, return_tensors='pt')
            tokens = tokens['input_ids']
            tokens = tokens.type(torch.long).to(device)
            tokenizados.append(tokens)
    return tokenizados

def TransformarNotasEmVetor(textos, notas):
    novas_notas = []
    for indice in range(len(textos)):
            novas_notas.append(torch.tensor(int(notas[indice]), dtype=torch.long).unsqueeze(0).to(device))
    return novas_notas

novos_inputs = TransformarTextoEmInput(texto_treinamento)
print(len(novos_inputs))
novas_notas = TransformarNotasEmVetor(texto_treinamento, nota_treinamento)
print(len(novas_notas))

3198
3198


In [5]:
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self): 
        super(CustomModel,self).__init__() 
        self.model = AutoModel.from_pretrained(modelo).to(device)
        self.classifier = nn.Sequential(nn.Linear(768,6), nn.Softmax(dim=0))
    def forward(self, input_ids):
        #print("Input: ", input_ids.size())
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids)
            #outputs = torch.squeeze(outputs['pooler_output'])
            outputs = outputs.last_hidden_state[0][0]
        logits = self.classifier(outputs) 
        return logits
model2 = CustomModel().to(device)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def treinar(model, inputs, target):
    #loss_fn = torch.nn.MSELoss()
    #print(pesos)
    loss_fn = nn.CrossEntropyLoss(weight=pesos)
    #loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    vetor_loss = []
    optimizer.zero_grad()
    count = 0
    for index in range(len(inputs)):
        count += 1
        output = model(inputs[index])
        output = output.unsqueeze(0)
        #r = torch.tensor(target[index]).unsqueeze(0).long()
        #print(target[index])
        #r = torch.tensor(target[index], dtype=torch.long)
        r = target[index]
        #print(r)
        #print(output, r)
        loss = loss_fn(output, r)
        vetor_loss.append(loss.item())
        loss.backward()
        if (count == 1):    
            optimizer.step()
            optimizer.zero_grad()
            count = 0
    loss_tmp = sum(vetor_loss)/len(vetor_loss)
    print("Loss media: ", loss_tmp)
    
def TokenizarUmParagrafo(paragrafo):
    tokens = tokenizer.encode_plus(paragrafo, add_special_tokens=True, truncation=True, return_tensors='pt')
    tokens = tokens['input_ids']
    tokens = tokens.type(torch.int64).to(device)
    return tokens

def acuracia_classe(respostas, gold_labels):
    classes_chutadas = [0]*6
    classes_acertadas = [0]*6
    classes_certas = [0]*6
    porcentagem_final = []
    for r, gl in zip(respostas, gold_labels):
        #print(gl)
        classes_certas[int(gl)] += 1
        classes_chutadas[int(r)] += 1
        if (r == gl):
            classes_acertadas[int(r)] += 1
    for acertos, certo in zip(classes_acertadas, classes_chutadas):
        if certo != 0:
            porcentagem_final.append( float(acertos) / certo )
        else:
            porcentagem_final.append(0.0)       
    print("Chutei: ", classes_chutadas, sum(classes_chutadas))
    print("Acertei: ", classes_acertadas, sum(classes_acertadas))
    print("Dist: ", classes_certas, sum(classes_certas))
    return porcentagem_final

def testar(model, inputs, target):
    respostas = []
    for index in range(len(inputs)):
        #notas_parciais = []
        with torch.no_grad():
            tokenizado = TokenizarUmParagrafo(inputs[index])
            output = model(tokenizado)
            nota = output.cpu().detach().numpy()
            #notas_parciais.append(nota)
            #print(len(notas_parciais))
            #nota_final = np.rint(np.average(notas_parciais))
            nota_final = np.argmax(nota)
        respostas.append(nota_final)
    #print(respostas)
    print("QWK: ", metrics.cohen_kappa_score(target, respostas, weights='quadratic'))
    #print("RMSE: ", metrics.mean_squared_error(target, respostas, squared=False))
    print("MSE: ", metrics.mean_squared_error(target, respostas, squared=True))
    print("Porcentagem das classes: ", acuracia_classe(respostas, target))
    print("Total acc: ", metrics.accuracy_score(target, respostas))

In [7]:
for i in range(5):
    print("Iteracao ", i+1)
    treinar(model2, novos_inputs, novas_notas)
    print("-- Treinamento: ")
    testar(model2, texto_treinamento, nota_treinamento)
    print("--Validacao:")
    testar(model2, texto_valid, nota_valid)
    print("--Teste:")
    testar(model2, texto_teste, nota_teste)

Iteracao  1
Loss media:  1.540511811726387
-- Treinamento: 
QWK:  0.33102979110387853
MSE:  0.8136335209505942
Chutei:  [0, 0, 0, 2141, 1057, 0] 3198
Acertei:  [0, 0, 0, 1012, 701, 0] 1713
Dist:  [74, 60, 478, 1202, 1197, 187] 3198
Porcentagem das classes:  [0.0, 0.0, 0.0, 0.47267631947687994, 0.663197729422895, 0.0]
Total acc:  0.5356472795497186
--Validacao:
QWK:  0.33777883387293206
MSE:  0.8527696793002916
Chutei:  [0, 0, 0, 438, 248, 0] 686
Acertei:  [0, 0, 0, 210, 146, 0] 356
Dist:  [20, 13, 101, 269, 244, 39] 686
Porcentagem das classes:  [0.0, 0.0, 0.0, 0.4794520547945205, 0.5887096774193549, 0.0]
Total acc:  0.5189504373177842
--Teste:
QWK:  0.32476294228297
MSE:  0.8192419825072886
Chutei:  [0, 0, 0, 459, 227, 0] 686
Acertei:  [0, 0, 0, 212, 149, 0] 361
Dist:  [15, 6, 110, 240, 264, 51] 686
Porcentagem das classes:  [0.0, 0.0, 0.0, 0.46187363834422657, 0.6563876651982379, 0.0]
Total acc:  0.5262390670553936
Iteracao  2
Loss media:  1.5086961248429438
-- Treinamento: 
QWK:  0.

Aqui farei o baseline mais simples de todos: dizer o índice mais comum

In [8]:
def learn_most_common_index(novas_notas):
    indices = [0,0,0,0,0,0]
    for i in novas_notas:
        indices[int(i)] += 1
    maior_valor = -1
    maior_indice = -1
    print(indices)
    for i in range(6):
        if(indices[int(i)] > maior_valor):
            maior_valor = indices[int(i)]
            maior_indice = i
    return maior_indice

def test_most_common_index(index, notas):
    acertos = 0
    respostas = []
    for i in notas:
        respostas.append(index)
    print("QWK1: ", m.kappa(notas, respostas, weights='quadratic'))
    print("QWK2: ", metrics.cohen_kappa_score(notas, respostas, weights='quadratic'))
    print("RMSE: ", metrics.mean_squared_error(notas, respostas, squared=False))
    print("Acc: ", metrics.accuracy_score(notas, respostas))
    #return (acertos/len(notas))

In [9]:
indice = learn_most_common_index(nota_treinamento)
print("Treinamento:")
test_most_common_index(indice, nota_treinamento)
print("Teste:")
test_most_common_index(indice, nota_teste)
print("Validacao:")
test_most_common_index(indice, nota_valid)

[74, 60, 478, 1202, 1197, 187]
Treinamento:
QWK1:  0.0
QWK2:  0.0
RMSE:  1.0202759930228251
Acc:  0.3758599124452783
Teste:
QWK1:  0.0
QWK2:  0.0
RMSE:  1.036505679349426
Acc:  0.3498542274052478
Validacao:
QWK1:  0.0
QWK2:  0.0
RMSE:  1.0336890826227816
Acc:  0.39212827988338195
